In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_rank(url, session):
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        #словарик
        data = {
            'Name': '',
            'Geo': '',
            'World University Rankings 2026': '',
            'Arts and Humanities 2025': '',
            'Business and Economics 2025': '',
            'Medical and Health 2025': '',
            'Computer Science 2025': '',
            'Education Studies 2025': '',
            'Engineering 2025': '',
            'Law 2025': '',
            'Life Sciences 2025': '',
            'Physical Sciences 2025': '',
            'Social Sciences 2025': ''
        }
        
        #ищем название через селекторы в главном заголовке
        name = soup.select_one('#__next > main > div > div > div > div > div.css-1dwnmlj > div.desktopHeaderRef.css-wxa6cs > div.css-17ek16l > div > div > div.css-axsv8p > div:nth-child(1) > h1')
        if name:
            data['Name'] = name.get_text(strip=True)
        
        #из соседнего div берём всё, что есть о геоположении
        geo = soup.select_one('#__next > main > div > div > div > div > div.css-1dwnmlj > div.desktopHeaderRef.css-wxa6cs > div.css-17ek16l > div > div > div.css-axsv8p > div:nth-child(1) > div.css-2mldjl > div > span')
        if geo:
            data['Geo'] = geo.get_text(strip=True)
        
        #нужный контейнер лежит здесь
        cont = soup.select_one('#rankings > div.css-p2re59 > div > div > div > div > div > div.react-horizontal-scrolling-menu--wrapper > div.react-horizontal-scrolling-menu--inner-wrapper > div.react-horizontal-scrolling-menu--scroll-container')
        
        if cont:
            #список соответствий индексов и названий столбцов (чтоб проще ориентироваться в скрол меню)
            rankings_map = {
                0: 'World University Rankings 2026',
                1: 'Arts and Humanities 2025',
                2: 'Business and Economics 2025',
                3: 'Medical and Health 2025',
                4: 'Computer Science 2025',
                5: 'Education Studies 2025',
                6: 'Engineering 2025',
                7: 'Law 2025',
                8: 'Life Sciences 2025',
                9: 'Physical Sciences 2025',
                10: 'Social Sciences 2025'
            }
            
            #парсим элементы
            for i in range(11):
                item = cont.select_one(f'#item-{i}')
                if item:
                    #нужен спан с классом .css-13clqac
                    rank_el = item.select_one('button > div > h4 > span.css-13clqac')
                    if rank_el:
                        rank_text = rank_el.get_text(strip=True)
                        #извлекаем число без лишнего
                        rank_num = ''.join(filter(str.isdigit, rank_text))
                        if rank_num:
                            data[rankings_map[i]] = rank_num 
        
        return data
        
    except Exception as e:
        print(f"error {url}: {e}")
        return None

def main():
    input_file = 'the_rankings_2026_full.csv'
    output_file = 'parsed_rankings2500_2810.csv' 
    #здесь можно менять для сохранения в разные файлы параллельных записей
    
    try:
        df = pd.read_csv(input_file)
        #обрабатываем ошибки
    except FileNotFoundError:
        print(f'не входного файла')
        return
    except Exception as e:
        print(f'не читается файл: {e}')
        return
    
    #регулируем!
    start_i = 2500  #начало
    end_i = 2811   #конец
    end_i = min(end_i, len(df))

    
    #запросная сессия
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })
    
    # Список для хранения всех данных
    all_data = []
    
    for idx in range(start_i, end_i):
        url = df.iloc[idx]['URL']
        
        #проверка на пустоту
        if pd.isna(url) or str(url).strip() == '':
            continue
        
        print(f'сейчас на {idx+1}/{end_i}: {url}')
        
        #получаем данные
        university_data = parse_rank(url, session)
        
        if university_data:
            all_data.append(university_data)
            print(f"Имя получили")
        else:
            print(f'имя не получили')
        time.sleep(1)
    
    #создаём файл
    if all_data:
        output_df = pd.DataFrame(all_data)
        output_df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\nитог {output_file}")
    else:
        print('нет данных')

if __name__ == "__main__":
    main()